In [ ]:
# Takes the output of phase2assignment and builds segments based on the pairings made
# pairing: bunch of tuples (carry1,prep1); (carry2, prep1)
# segments: a list of sites starting at a prep site and ending at some carry in site
# no more than 2(?) carry ins per site
# have time windows come in here? 
# automatically put breakfasts on same segments and lunches on others? 

In [74]:
%run ./phase2assignment.ipynb
# Comment out in phase2assignment whatever you don't want re-printing here
# runs phase2 so that all of those outputs/functions are easily used here 

Total distance (km) = 120.52767157400005


In [ ]:
def demand_sum(carry_list):
    # Total demand of the sites in any given sized list
    demand = 0
    for c in carry_ins:
        demand = demand + carry_in_dict[c]
    return demand

In [6]:
vehicle_capacity = 800
max_segment_length = 2

In [7]:
# Get all unique prep site values into one list
unique_prep = []
for key in assignments.keys():
    prep = key[1]
    if prep not in unique_prep:
        unique_prep.append(prep)

In [76]:
# Make dict of 1:1 pairings into condensed dict where a prep set now points to a list of every carry in it serves
prep_list = {}
for p in unique_prep:
    prep_list[p] = []
    # key p now points to empty list
    for key in assignments.keys():
        if p == key[1]:
            prep_list[p].append(key[0])
            ## take k[0] and add it to list that p points to in prep_list

In [ ]:
def segment_optimization(prep, list_of_carry_ins):
    solver = pywraplp.Solver('SolveAssignmentProblemMIP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    dummies = [0] * 4
    for i in range(dummies):
        dummies[i] = prep + i # (makes them distinct from each other if not from every site)
        
    carry_in_site_number = len(list_of_carry_ins)
    prep_site_number = len(dummies)
    bools = {}
    
    for i in carry_in_site_number:
        for j in prep_site_number: 
            bools[(i, j)] = solver.BoolVar('bool[%i,%i]' % (i, j))
            
    # What to minimize? 
    solver.Minimize(solver.Sum([costs[(i,j)]*segments[(i,j)]
                                        for i in carry_in_site_number
                                        for j in prep_site_number]))
            
    for i in carry_in_site_number: # carry in can only be assigned to one dummy site
        solver.Add(solver.Sum([bools[(i,j)] for j in prep_site_number]) == 1)
        
    for j in prep_site_number: 
        # a dummy prep can have max 2 carry ins assigned to it
        solver.Add(solver.Sum([bools[(i,j)] for i in carry_in_site_number]) <= 2)
        
        # a segment cannot require more than the max vehicle capacity
        solver.Add(solver.Sum([bools[(i,j)]*carry_in_dict[i] for i in carry_in_site_number]) <= vehicle_capacity)
    # copy code from part 1
    # new constraints: 
    # for i in carry_in_site_number:
    # solver.Add(solver.Sum([bools[(i,j)] for j in prep_site_number]) == 1)
    # solver.Add(solver.Sum([bools[(i,j)] for i in carry_in_site_number]) <= 2)
    # segment cannot handle more than the vehicle capacity 
    # solver.Add(solver.Sum([bools[(i,j)]*carry_in_dict[i] for i in carryins]) <= vehicle_capacity)
    #dummies = {} 
    # declare solver
    # declare variables (similar to part 1)
    # declare constraints (vehicle capacity, number of sites)

In [77]:
prep_list

{419: [110, 1171],
 408: [158],
 301: [161, 509],
 455: [168, 605, 3830, 3831, 4880, 4881],
 190: [178, 252],
 150: [179],
 248: [192, 4990],
 412: [203, 223],
 464: [212, 436, 4991],
 405: [264],
 457: [328, 750],
 450: [394, 6040, 6041],
 415: [426],
 451: [438],
 218: [473, 4781, 4790, 5150, 5151],
 423: [477, 801, 2130, 2131, 4900, 4901],
 682: [497, 891, 4780, 6020, 6021],
 982: [516, 3160, 3161],
 275: [533],
 437: [999, 4791],
 424: [1170],
 971: [1810, 1811],
 981: [5220, 5221, 5320, 5321]}

In [59]:
## build segments for each of these keys
#constraints: vehicle capacity, number of segments
#objective: fewest possible segments

## make multiple 'prep' sites and build segments based on that 
# ex: for 423,4230,4231,4232,etc... 
# assign based on vehicle capacity & 

In [65]:
## If just one carry in site, leave as is

# for prep in prep_list.keys():
#    carry_ins = prep_list[prep]
#    if len(carry_ins) == 1:
#        final_segments[prep] = carry_ins

## If two, check distance, if significant, separate (significant distance = ?)
    # if len(carry_ins) == 2:
    #    if carry_in_dict[carry_ins[0]] + carry_in_dict[carry_ins[1]] >= vehicle_capacity:
    #      final_segments[prep] = [[carry_ins[0]],[carry_ins[1]]]
     #   else: do the string length comparisons 
    ## If different endings, group separately
    ##  str1 = str(carry_ins[0])
    ## len1 = len(str1)
    ##  str2 = str(carry_ins[1])
    ## len2 = len(str2)
        # if len1 == 4 and len2 == 4:
            # if str1[3] == str2[3]:
            # same segment
        # if len1 == 3:
            # len2 == 3:
                # both lunch
            #elif str2[3] == 1
                # both lunch
            #else: different
        # if len2 == 3:
            # if len1 == 4:
                # if str1[3] == 1: both lunch
                #else: different
            #else: same 
            
    ## both can be lunch, both can be breakfast, or they can be different (can be lunch if 4 dig ending with 1 or 3 dig)
        
    # if len(str(carry_ins[0])) == 4
    # if distance < whatever:
        # final_segments[prep] = [carry_ins] (makes the result a list of the two sites, which implies they're segmented together)
## for more than two, create dummy prep sites (4231,4232,4233,4234,etc)
## Assign up to two carry ins to each dummy site, considering vehicle constraints, whether breakfast/lunch, and proximity
## 

In [ ]:
## re-designed version of part i
## make a bunch of pseudo prep sites for all of the preps and assign carry ins to the pseudo preps to build segments
    # group breakfast and lunches together, unless they're v far apart 
## don't need to do if count of carry in < 3

In [70]:
# basically instead of optimizing I'm just building lists using logic
# unsure what to do if prep site has more than 2 carry in sites assigned
# would want it to go through the list of carry in sites and put them into segment lists based on demand and maybe time of day?
# like if 4 digits with a 1 at the end, lunch, so pair with other lunch, etc? 
# but there's an ambiguous number of segments 

#fake_segments = {}
#unassigned_dict = {}
#
#    elif len(carry_ins) == 2:
#        if carry_in_dict[carry_ins[0]] + carry_in_dict[carry_ins[1]] >= vehicle_capacity:
#            final_segments[prep] = [[carry_ins[0]],[carry_ins[1]]]
#    elif len(carry_ins) > 2: ## ambiguous number of lists, don't know how many segments we'll make? 
#        temp = []
#        seg = []
#        unassigned = []
#        for site in carry_ins:
#            if demand_sum(temp.append(site)) < vehicle_capacity:
#                seg.append(site)
#            else:
#                unassigned.append(site)
#            final_segments[prep] = seg
#            unassigned_dict[prep] = unassigned
 ## should be some sort of method that builds a list of carry ins               
            
        
        

            
                    

#print(final_segments)


In [78]:
str(1041)

'1041'

In [79]:
x = str(1041)

In [80]:
x[4]

IndexError: string index out of range

In [81]:
x[3]

'1'

In [82]:
dummy = []

In [83]:
len(dummy)

0